In [ ]:
version = "REPLACE_PACKAGE_VERSION"

# Assignment 4 - Tree-based classification; Synthesis Project

This assignment has two parts: the first part reviews tree-based classification, and the second part is a synthesis project.

In [ ]:
# import some necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Suppress all warnings
import warnings
warnings.filterwarnings('ignore')

## Part 1: Classification. Predicting landscape types from satellite data.

Machine learning is being used increasingly to help fight climate change and promote sustainable land use. For this section of the assignment we will be working with a dataset derived from geospatial data, and our classification goal will be to predict the type of landscape visible by satellite. With an accurate classifier, we could monitor changes in land use and characteristics over time on a global scale.

Summary of the dataset: "The dataset is derived from two sources: 1) Landsat time-series satellite imagery from the years 2014-2015, and 2) crowdsourced georeferenced polygons with land cover labels obtained from OpenStreetMap. The crowdsourced polygons cover only a small part of the image area, and are used to extract training data from the image for classifying the rest of the image. The main challenge with the dataset is that both the imagery and the crowdsourced data contain noise (due to cloud cover in the images and inaccurate labeling/digitizing of polygons).

The 'landsat_training.csv' file contains the training data for classification. Do not use this file to evaluate classification accuracy because it contains noise (many class labeling errors).

The 'landsat_testing.csv' file contains testing data to evaluate the classification accuracy. Do not use this file to train the classifier. This file does not contain any class labeling errors.

Attribute Information (columns):

`class`: The land cover class (impervious, farm, forest, grass, orchard, water) **Note: this is the target variable to classify.**

`max_ndvi`: the maximum NDVI (normalized difference vegetation index) value derived from the time-series of satellite images.

`20150720_N - 20140101_N` : NDVI values extracted from satellite images acquired between January 2014 and July 2015, in reverse chronological order (dates given in the format yyyymmdd)."

This is a multi-class classification problem with six classes. Thus, you will want to examine both micro- and macro-averaged evaluation scores across all six classes, e.g. to see if there are landscape types that are more difficult than others to classify correctly.


Use the following code to read in the data files. Note that we convert the strings giving the landscape types (the "class" column to be predicted) to numeric labels using `LabelEncoder`.

In [ ]:
df_train = pd.read_csv('assets/landsat_training.csv')
df_test  = pd.read_csv('assets/landsat_testing.csv')

from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df_train['class_code'] = enc.fit_transform(df_train['class'])
df_test['class_code']  = enc.transform(df_test['class'])

class_names = enc.classes_
print(class_names)

X_train = df_train.iloc[:,1:28]
y_train = df_train['class_code']

X_test = df_test.iloc[:,1:28]
y_test = df_test['class_code']


### Question 1. Basic decision trees (10 points)

Using `X_train` and `y_train` from the preceeding cell, train a `DecisionTreeClassifier` with default parameters and `random_state=0`. What are the 5 most important features found by the decision tree?

*Your function should return a tuple of length 5 of the feature names in descending order of importance.*

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def answer_one():
    
    clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
    
    top_five = clf.feature_importances_.argsort()[::-1][:5]   
    features = X_train.columns[top_five]
    
    return tuple(features)

In [ ]:
# Autograder tests

stu_ans = answer_one()

assert isinstance(stu_ans, tuple), "Q1: Your function should return a tuple. "
assert len(stu_ans) == 5, "Q1: The length of your returned tuple should be 5. "
assert all([isinstance(item, str) for item in stu_ans]), "Q1: Your tuple should only contain strings. "


del stu_ans

### Question 2.  (10 points) 

One very important decision tree parameter that can have a significant effect on how much the data are overfit (or not) is the *maximum tree depth*. Increasing maximum tree depth may increase accuracy by allowing more complex decision rules involving more features, but also increases the potential for overfitting with deeper trees.

Examine the effect of maximum tree depth on training and test set accuracy for a basic decision tree on this dataset. 
Return a list of `(train_score, test_score)` tuples for each possible maximum tree depth from 1 to 10.

Use `random_state=0` for any calls involving randomization.

In [ ]:
def answer_two():  

    results = []

    for i in range(1, 11):

        clf = DecisionTreeClassifier(max_depth = i, random_state=0).fit(X_train, y_train)

        test_score = clf.score(X_test, y_test)
        train_score = clf.score(X_train, y_train)

        results.append((train_score, test_score))

    return results

In [ ]:
# Autograder tests

stu_ans = answer_two()

assert isinstance(stu_ans, list), "Q2: Your function should return a list. "
assert len(stu_ans) == 10, "Q2: The length of your returned list should be 10. "
assert all([isinstance(item, tuple) for item in stu_ans]), "Q2: Your list should only contain tuples. "

del stu_ans

Uncommenting the code below, take a look at the classification report for this GBDT model using the *test* set to evaluate. It is instructive to compare these results with the results from previous classifiers. 

In [ ]:
# Remember to comment them out before submitting the notebook

# test_accs = [t for _, t in answer_two()]
# best_tree_model = tree.DecisionTreeClassifier(random_state=0, max_depth=np.argmax(test_accs) + 1).fit(X_train, y_train)
# y_pred = best_tree_model.predict(X_test)
# print(classification_report(y_test, y_pred, target_names=class_names))
# del test_accs, best_tree_model, y_pred

### Question 3. Gradient-boosted decision trees (10 points)

We're now going to apply GBDT to the same dataset to compare its performance with ordinary decision trees. Train a default GBDT and output the training and test scores (accuracy) in that order, as a float 2-tuple. (Use `random_state=0` for any randomized operations.)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

def answer_three():
    
    clf = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
    
    test_score = clf.score(X_test, y_test)
    train_score = clf.score(X_train, y_train)
    
    results = train_score, test_score
    
    return results

In [ ]:
#Autograder tests

stu_ans = answer_three()

assert isinstance(stu_ans, tuple), "Q3a: Your function should return a tuple. "
assert len(stu_ans) == 2, "Q3a: The length of your returned tuple should be 2. "
assert all([isinstance(item, float) for item in stu_ans]), "Q3a: Your tuple should only contain floats. "

del stu_ans

## Part 2: Synthesis project

### Question 1. City of Detroit Prediction Problem (70 points)

The synthesis project is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - building deterioration. [Blight violations](https://detroitmi.gov/departments/department-appeals-and-hearings/blight-ticket-information) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. Some related datasets of interest include:

* [Building Permits](https://data.detroitmi.gov/datasets/building-permits)
* [Trades Permits](https://data.detroitmi.gov/datasets/trades-permits)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/datasets/improve-detroit-issues)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/datasets/dpd-citizen-complaints)
* [Parcel Data](https://data.detroitmi.gov/datasets/parcels-2)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will receive full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

def blight_model():
    
    # Load .csv's
    initial_train_data = pd.read_csv('assets/train.csv')
    initial_test_data = pd.read_csv('assets/test.csv')
    addresses = pd.read_csv('assets/addresses.csv')
    latlons = pd.read_csv('assets/latlons.csv')
    
    # Join address and latlons to train and test data
    train_data = pd.merge(initial_train_data, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    test_data = pd.merge(initial_test_data, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    
    # Filter necessary columns
    cols = ['fine_amount', 'discount_amount', 'judgment_amount', 'lat', 'lon', 'compliance']
    train_data = train_data[cols]
    test_data = test_data[cols[:-1]]
    
    # Filter null compliance rows
    train_data = train_data[train_data.compliance.notnull()]
    
    # Fill null data points
    train_data = train_data.fillna(0)
    test_data = test_data.fillna(0)
    
    # Create x/y train/test
    X_train = train_data.iloc[:, :-1]
    y_train = train_data['compliance']
    X_test = test_data
    
    # Train model
    model = RandomForestClassifier(random_state = 42)
    grid = {'n_estimators': [100], 'max_depth': [10]}
    grid_models = GridSearchCV(model, param_grid=grid, scoring='roc_auc')
    grid_models.fit(X_train, y_train)

    # Predict scores
    y_scores = grid_models.best_estimator_.predict_proba(X_test)

    rec = pd.DataFrame(y_scores[:, 1], initial_test_data.ticket_id)
    
    return rec.squeeze()

In [ ]:
# Autograder tests - sanity checks; no points for passing this cell

stu_ans = blight_model()

assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series. "
assert len(stu_ans) == 61001, "Your series is of incorrect length. "


In [ ]:
# Hidden autograder tests - whether AUC >= 0.55


In [ ]:
# Hidden autograder tests - whether AUC >= 0.6


In [ ]:
# Hidden autograder tests - whether AUC >= 0.65


In [ ]:
# Hidden autograder tests - whether AUC >= 0.7


In [ ]:
# Hidden autograder tests - whether AUC >= 0.75
